In [15]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bsp
import matplotlib
import matplotlib.pyplot as plt
import sqlite3
import re
from datetime import datetime, timedelta,date
import sys
import time
import click
from selenium.webdriver.common.action_chains import ActionChains
import sqlite3

In [16]:
def getData(d): # 데이터베이스에 데이터 집어넣기
    dbname='perfume.db'
    with sqlite3.connect(dbname) as conn:
        cursor=conn.cursor()
        sql='INSERT INTO perfumegra (brand_name,name,age,gender,grade,review1) VALUES(?,?,?,?,?,?)' 
        res=cursor.execute(sql,d)
        print('.',end='')

In [19]:
from selenium import webdriver
from bs4 import BeautifulSoup as bsp
from selenium.webdriver.chrome.options import Options

CHROMEDRIVER_PATH = 'C:/asi/project/chromedriver_win32 (3)/chromedriver' # Windows는 chromedriver_78.exe
WINDOW_SIZE = "1920,1080"
chrome_options = Options()
chrome_options.add_argument(f"--window-size={ WINDOW_SIZE }")
chrome_options.add_argument('Content-Type=application/json; charset=utf-8')
driver = webdriver.Chrome( executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options )
url = ('https://perfumegraphy.com')
driver.get(url)
#driver = webdriver.Chrome('C:/asi/project/chromedriver_win32 (3)/chromedriver')
#url = ('https://perfumegraphy.com/product/list.html?cate_no=201')
#driver.get(url)
driver.implicitly_wait(3)
driver.find_element_by_xpath('//*[@id="slideCateList"]/div/div').click()
driver.implicitly_wait(3)
time.sleep(3)
html = driver.page_source
src = bsp(html,'html.parser')
brands=src.find_all(attrs={'class':'-brand'})
for brand in brands:
    brand_url="https://perfumegraphy.com"+brand.find('a')['href']
    brand_name=brand.find('a').text
    driver.get(brand_url)
    driver.implicitly_wait(3)
    html = driver.page_source
    src = bsp(html,'html.parser')
    perurls = src.select('li.xans-record- > div')
    for perurl in perurls:
        urls2 = perurl.select('div.prdImg > a')[0]
        homepage = urls2['href']
        productids = homepage.split('/')
        nums = productids[3]
        product_name=perurl.find(attrs={'class':'name'}).text.replace('상품명 : ','').replace('/','')
        product_name = re.sub('\d*?ml','',product_name).strip().replace('NEW','').replace('60-','').replace('30-','')
        product_name = re.sub('new.*?','',product_name).strip().replace('-','').replace('75125','')
        product_name = re.sub('x.*?','',product_name).strip()
        product_name = re.sub('\*.*?','',product_name).strip()
        product_name = re.sub('\+.*?','',product_name).strip()
        if product_name.count('비누') >0:
            pass
        elif product_name.count('캔들') >0:
            pass
        elif product_name.count('5종') >0:
            pass
        else:
            i = 1
            while True:
                url='http://review4.cre.ma/perfumegraphy.com/products/reviews?app=0&iframe=1&iframe_id=crema-product-reviews-1&page={0}&parent_url=https%3A%2F%2Fperfumegraphy.com%2Fproduct%2F%25EA%25B5%25AC%25EC%25B0%258C-%25EB%25B8%2594%25EB%25A3%25B8-%25EC%2598%25A4-%25EB%2593%259C-%25ED%258D%25BC%25ED%2593%25B8-3050100ml-%25ED%2585%258C%25EC%258A%25A4%25ED%2584%25B0%2F1469%2Fcategory%2F201%2Fdisplay%2F1%2F&product_code={1}&secure_device_token=V281c8ef9fe71647c61b7e5908c3e2cd131479b904aa9876805c46f46e9cb0b841&widget_env=100&widget_id=32&widget_style='.format(i,nums)    
                headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}
                res = requests.get(url,headers=headers)
                src = res.content.strip()
                htmls = bsp(src, 'html.parser')
                time.sleep(1)
                try:
                    review_all = htmls.find_all(attrs={'class':'review products_reviews_board__tr'})
                    for review2 in review_all:
                        grade = review2.select('span.star')
                        review_ = review2.select('div.products_reviews_board_review__message_expanded')[0].text.strip()
                        #print(review_)
                        review1 = re.sub('\([^)]*\)','',review_)
                        #print(review1)
                        if len(review2.select('div.review_options > div.review_option')) == 3:
                            #print(review2.select('div.review_options > div.review_option'))
                            #print(review2.select('div.review_option__title'))
                            if review2.select('div.review_options > div.review_option > div.review_option__title')[0].text == '연령대':
                                age = review2.select('div.review_options > div.review_option > div.review_option__content')[0].text
                                #print(age)
                                gender = review2.select('div.review_options > div.review_option > div.review_option__content')[1].text
                                #print(age,gender)
                        elif len(review2.select('div.review_options > div.review_option')) == 2:
                            if review2.select('div.review_options > div.review_option > div.review_option__title')[0].text == '연령대':  
                                age = review2.select('div.review_options > div.review_option > div.review_option__content')[0].text
                                #print(age)
                                if review2.select('div.review_options > div.review_option > div.review_option__title')[1].text == '성별':
                                    gender = review2.select('div.review_options > div.review_option > div.review_option__content')[1].text
                                else:
                                    gender = ''
                            elif review2.select('div.review_options > div.review_option > div.review_option__title')[0].text == '성별':
                                gender = review2.select('div.review_options > div.review_option > div.review_option__content')[1].text
                                age = ''    
                        elif len(review2.select('div.review_options > div.review_option')) == 1:
                            if review2.select('div.review_options > div.review_option > div.review_option__title')[0].text == '연령대':
                                age = review2.select('div.review_options > div.review_option > div.review_option__content')[0].text
                                gender = ''
                            elif review2.select('div.review_options > div.review_option > div.review_option__title')[0].text == '성별':
                                gender = review2.select('div.review_options > div.review_option > div.review_option__content')[0].text
                                age = ''
                            else :
                                age = ''
                                gender = ''       
                        else:
                            age = ''
                            gender = ''  
                        print('.',end='')
                        d = (brand_name,product_name,age,gender,len(grade),review1)
                        getData(d)
                        time.sleep(1)
                    if htmls.select('a.pagination__button'):
                        print(i)
                        i+=1
                    else :
                        break
                except:
                    pass

<ipython-input-19-04f36f91affc>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome( executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options )


................................................1
..........2
..........3
....4
..........1
..........2
..3
..........1
....2
........................1
..2
..........1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........9
..........10
..........11
..........12
..........13
..........14
..........15
..........16
..........17
..........18
..........19
..........20
..........21
..........22
..........23
..........24
..........25
..........26
..........27
..........28
..........29
..........30
..........31
..........32
..........33
..........34
..........35
..........36
..........37
..........38
..........39
..........40
..........41
..........42
..........43
..........44
..........45
..........46
..........47
..........48
..........49
..........50
..........51
..........52
..........53
..........54
..........55
..........56
..57
..........1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........9
..........10

..........34
..........35
..........36
..........37
..........38
..........39
..........40
..........41
..........42
..........43
..........44
..........45
..........46
..........47
..........48
..........49
..........50
..........51
..........52
..........53
..........54
..........55
..........56
..........57
..........58
..........59
..........60
..........61
..........62
..........63
..........64
..65
..........1
..........2
..........3
..........4
..........5
..........6
..........1
..........2
..........3
..........4
..........5
..........1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........9
..........10
..........11
..........12
..........13
..........14
..........15
..........16
..........17
..........18
..........19
..........20
..........21
..........22
..........23
......24
..........1
..2
..........1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........9
..........10
..........11
..........1

..........7
..........8
..........9
..........10
......11
..........1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........9
........10
..........1
....2
..........1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........9
..10
..........1
..........2
..........3
..4
................1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
........9
..........1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........9
..........10
..........11
..........12
..........13
..........14
....15
............1
..........2
..........3
..........4
....5
..........1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
......9
..........1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........9
..........10
..11
..........1
..........2
..........3
..........4
..........5
..........6
..

..........14
..........15
..........16
..........17
..........18
..........1
......2
..........1
..........2
..........3
..........4
..5
..........1
..........2
..........3
..........4
..5
..........1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........9
..........10
..........11
..........12
....13
..........1
..........2
..........3
........4
..........1
..2
..........1
..........2
..........3
..........4
........5
..................1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........9
..........10
..........11
..........12
..........13
..........14
..........15
........16
..............1
..........2
..........1
..........2
..........1
..........2
..................1
......2
..............1
......2
..............................1
..........2
....3
....................1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........9
..........10
..........11
..........

..........4
..........5
..........6
..........7
..........8
..........9
..........10
..........11
..........12
..........1
......2
......................1
..........2
..........3
......4
..........1
....2
....................1
..........2
......3
..........1
..........2
..........3
..........4
..........5
............................1
..........2
................1
..........2
..........3
............................1
..........2
..........3
..4
..............1
....2
........................1
..........2
..........3
..........4
..........5
..........6
..........7
....8
..........1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........9
..........10
..........11
..........12
..........13
..........14
..........15
..........16
..........17
..18
..........1
..........2
............1
..........2
..........3
..........4
..........5
..........6
..7
..........1
..........2
..........3
..........4
..........5
..........6
..........7
..........8
..........

In [ ]:
perurls = src.select('li.xans-record- > div')
for perurl in perurls:
    urls2 = perurl.select('div.prdImg > a')[0]
    homepage = urls2['href']
    productids = homepage.split('/')
    nums = productids[3]
    name_ = productids[2].replace('-','').replace('딥디크','').strip()
    name = re.sub('[a-zA-Z0-9]','',name_).strip()
    brand_name_ = productids[2].replace('-','').strip()
    brand_name = brand_name_[:3]
    i = 1
    while True:
        url='http://review4.cre.ma/perfumegraphy.com/products/reviews?app=0&iframe=1&iframe_id=crema-product-reviews-1&page={0}&parent_url=https%3A%2F%2Fperfumegraphy.com%2Fproduct%2F%25EA%25B5%25AC%25EC%25B0%258C-%25EB%25B8%2594%25EB%25A3%25B8-%25EC%2598%25A4-%25EB%2593%259C-%25ED%258D%25BC%25ED%2593%25B8-3050100ml-%25ED%2585%258C%25EC%258A%25A4%25ED%2584%25B0%2F1469%2Fcategory%2F201%2Fdisplay%2F1%2F&product_code={1}&secure_device_token=V281c8ef9fe71647c61b7e5908c3e2cd131479b904aa9876805c46f46e9cb0b841&widget_env=100&widget_id=32&widget_style='.format(i,nums)    
        headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36'}
        res = requests.get(url,headers=headers)
        src = res.content.strip()
        htmls = bsp(src, 'html.parser')
        time.sleep(1)
        try:
            review_all = htmls.find_all(attrs={'class':'review products_reviews_board__tr'})
            #print(review_all)
            #print('-'*100)
            for review2 in review_all:
                grade = review2.select('span.star')
                #print(len(scores))
                review_ = review2.select('div.products_reviews_board_review__message_expanded')[0].text.strip()
                #print(review_)
                review1 = re.sub('\([^)]*\)','',review_)
                #print(review1)
                if len(review2.select('div.review_options > div.review_option')) == 3:
                    #print(review2.select('div.review_options > div.review_option'))
                    #print(review2.select('div.review_option__title'))
                    if review2.select('div.review_options > div.review_option > div.review_option__title')[0].text == '연령대':
                        age = review2.select('div.review_options > div.review_option > div.review_option__content')[0].text
                        #print(age)
                        gender = review2.select('div.review_options > div.review_option > div.review_option__content')[1].text
                        #print(age,gender)
                elif len(review2.select('div.review_options > div.review_option')) == 2:
                    if review2.select('div.review_options > div.review_option > div.review_option__title')[0].text == '연령대':  
                        age = review2.select('div.review_options > div.review_option > div.review_option__content')[0].text
                        #print(age)
                        if review2.select('div.review_options > div.review_option > div.review_option__title')[1].text == '성별':
                            gender = review2.select('div.review_options > div.review_option > div.review_option__content')[1].text
                        else:
                            gender = ''
                    elif review2.select('div.review_options > div.review_option > div.review_option__title')[0].text == '성별':
                        gender = review2.select('div.review_options > div.review_option > div.review_option__content')[1].text
                        age = ''    
                elif len(review2.select('div.review_options > div.review_option')) == 1:
                    if review2.select('div.review_options > div.review_option > div.review_option__title')[0].text == '연령대':
                        age = review2.select('div.review_options > div.review_option > div.review_option__content')[0].text
                        gender = ''
                    elif review2.select('div.review_options > div.review_option > div.review_option__title')[0].text == '성별':
                        gender = review2.select('div.review_options > div.review_option > div.review_option__content')[0].text
                        age = ''
                    else :
                        age = ''
                        gender = ''       
                else:
                    age = ''
                    gender = ''  
                print(brand_name)
                print(name)
                print(age)
                print(gender)
                print(len(grade))
                print(review1)
                print('*'*50)
                d = (brand_name,name,age,gender,len(grade),review1)
                getData(d)
                print(d)
                time.sleep(1)
            if htmls.select('a.pagination__button'):
                print(i)
                i+=1
            else :
                break
        except:
            pass

딥디크
오데썽오드뚜왈렛


5
친구 생일선물로 사줬어요 포장도 이쁘게 하고 마음에 드네요 ㅎㅎ 다음에 또 여기서 살게요 ㅎㅎ
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '친구 생일선물로 사줬어요 포장도 이쁘게 하고 마음에 드네요 ㅎㅎ 다음에 또 여기서 살게요 ㅎㅎ')
딥디크
오데썽오드뚜왈렛
30대
여
5
향수를 너무도 사랑하는 나딥디크... 첫 구 매..허브풀잎같은 알싸쌉싸름한향이강하다..ㅜ실 패. .
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '30대', '여', 5, '향수를 너무도 사랑하는 나딥디크... 첫 구 매..허브풀잎같은 알싸쌉싸름한향이강하다..ㅜ실 패. .')
딥디크
오데썽오드뚜왈렛
20대
여
5
아직 뿌려보진않았지만향은 괜찮은거같아용
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '20대', '여', 5, '아직 뿌려보진않았지만향은 괜찮은거같아용')
딥디크
오데썽오드뚜왈렛


5
도손도 좋아해서 사봤는데 향 너무좋음
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '도손도 좋아해서 사봤는데 향 너무좋음')
딥디크
오데썽오드뚜왈렛


5
향은 첨 맡아본향인데 호불호가있어서 걱정했는데 독하진않고 괜찮은거같아요
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '향은 첨 맡아본향인데 호불호가있어서 걱정했는데 독하진않고 괜찮은거같아요')
1
딥디크
오데썽오드뚜왈렛
20대
남
5
빠르게 잘받았습니다 향 좋아요!!
*******************************************

딥디크
오데썽오드뚜왈렛


5
만족아이유향수라서 아이유냄새맡을라고삿은데 너무조아요
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족아이유향수라서 아이유냄새맡을라고삿은데 너무조아요')
딥디크
오데썽오드뚜왈렛


5
만족나의 최애향수너무 좋아요
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족나의 최애향수너무 좋아요')
딥디크
오데썽오드뚜왈렛


5
향은 좋은데 지속력이 정말 짧아요ㅠ
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '향은 좋은데 지속력이 정말 짧아요ㅠ')
딥디크
오데썽오드뚜왈렛


5
만족동생과 여자친구 선물했는데 향이 너무 좋아서 계속 맡게되네요 다음에 또시킬께요
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족동생과 여자친구 선물했는데 향이 너무 좋아서 계속 맡게되네요 다음에 또시킬께요')
9
딥디크
오데썽오드뚜왈렛


5
보통배송빠르고 포장 꼼꼼합니다
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '보통배송빠르고 포장 꼼꼼합니다')
딥디크
오데썽오드뚜왈렛


5
만족생각했던향은 아닌거같아요 풀향이 많이 나는데 여름엔 괜찮을거같아요
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족생각했던향은 아닌거같아요 풀향이 많이 나는데 여름엔 괜찮을거같아요')
딥디크
오데썽오드뚜왈렛


5
만족향 좋아요 !!!!!!! 다 쓰면 또 구매하려구요

딥디크
오데썽오드뚜왈렛


5
만족잔향이 아주 좋습니다.
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족잔향이 아주 좋습니다.')
17
딥디크
오데썽오드뚜왈렛


5
만족좋아하는 향이에여 저렴하게 구입했습니다~
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족좋아하는 향이에여 저렴하게 구입했습니다~')
딥디크
오데썽오드뚜왈렛


5
만족엄마선물로 드렸어요ㅎㅎ 배송 빠른편이에요
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족엄마선물로 드렸어요ㅎㅎ 배송 빠른편이에요')
딥디크
오데썽오드뚜왈렛


5
잘 받았어요
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '잘 받았어요')
딥디크
오데썽오드뚜왈렛
30대
여
5
매우좋아요ㅋㅋ
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '30대', '여', 5, '매우좋아요ㅋㅋ')
딥디크
오데썽오드뚜왈렛


5
만족100은 너무 많아서 50 샀는데 양도 적당하고 냄새도 진짜 좋아요ㅠㅠ
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족100은 너무 많아서 50 샀는데 양도 적당하고 냄새도 진짜 좋아요ㅠㅠ')
18
딥디크
오데썽오드뚜왈렛


5
만족여자친구 선물 해줬는데 주변에서 향이 엄청 좋다고 하네욤
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '

23
딥디크
오데썽오드뚜왈렛
20대
남
5
평소 366님의 블로그를 자주 찾아보다가 오렌지 비누 향이란 말을 듣고 궁금해 사게 되었습니다 착 향을 해보니 정말 그 쌉싸름한 오랜 지향과 스파이시한 향이 나네요 미들 노트부터는 정말 좋아하는 향이 나네요 역시 366님의 표현력은 정확하네요. 센츠에서 나는 향도 취처입니다 ㅋㅋ 조만간 또 지르러 오겠습니다
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '20대', '남', 5, '평소 366님의 블로그를 자주 찾아보다가 오렌지 비누 향이란 말을 듣고 궁금해 사게 되었습니다 착 향을 해보니 정말 그 쌉싸름한 오랜 지향과 스파이시한 향이 나네요 미들 노트부터는 정말 좋아하는 향이 나네요 역시 366님의 표현력은 정확하네요. 센츠에서 나는 향도 취처입니다 ㅋㅋ 조만간 또 지르러 오겠습니다')
딥디크
오데썽오드뚜왈렛
20대
남
5
음 냄새는 좋은데 음.. 모르겟음 남자꺼를
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '20대', '남', 5, '음 냄새는 좋은데 음.. 모르겟음 남자꺼를')
딥디크
오데썽오드뚜왈렛


5
만족여친 선물로 줬는데 너무 좋아해요
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족여친 선물로 줬는데 너무 좋아해요')
딥디크
오데썽오드뚜왈렛


5
만족빠른배송!!!선물할껀데 배송빨라서 좋았어요 향 너무너무 좋아요
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족빠른배송!!!선물할껀데 배송빨라서 좋았어요 향 너무너무 좋아요')
딥디크
오데썽오드뚜왈렛
30대
여
5
딥디크 향 중에서 가장 좋아하는 향이에요.퍼퓸그라피에 사고 싶은 향수

딥디크
오데썽오드뚜왈렛


5
만족동생 졸업선물로 주문했는데 포장도 깔끔하고 좋네요
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족동생 졸업선물로 주문했는데 포장도 깔끔하고 좋네요')
딥디크
오데썽오드뚜왈렛


5
만족향이 너무좋아요 배송도 빨라 아주 만족합니다
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족향이 너무좋아요 배송도 빨라 아주 만족합니다')
31
딥디크
오데썽오드뚜왈렛


5
만족배송도 빠르고 포장도 깔끔합니당 감사해요
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족배송도 빠르고 포장도 깔끔합니당 감사해요')
딥디크
오데썽오드뚜왈렛
20대
남
5
시향은 못해봤었는데 추천글에서 제일 마음에 들어서 구매 했어요!! 시트러스향 너무 좋네요 ㅎㅎ 잔향도 마음에 들구요 ㅎㅎ
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '20대', '남', 5, '시향은 못해봤었는데 추천글에서 제일 마음에 들어서 구매 했어요!! 시트러스향 너무 좋네요 ㅎㅎ 잔향도 마음에 들구요 ㅎㅎ')
딥디크
오데썽오드뚜왈렛
10대
남
5
신세계 강남에서 시향해보고 반해서 안 지를 수가 없었던 오데썽입니다! 탑 노트의 상큼함이 빠지고 이어지는 고급스러운 비누향이 아주 예술입니다. 10대라 부담스럽지 않은 데일리 향수를 찾고 있었는데 잘 산 것 같네요.화요일 11시에 주문했더니 바로 배송 준비로 이어지더니 수요일에 오더군요. 배송 또한 굉장히 만족스럽습니다. ㅎㅎㅎ샘플도 굉장히 많이 챙겨주셨어요. 3개씩이나, 거기다 제가 남자 소비자라는 걸 고려해주셔서 남성용으로 주셨는데 한 가지 아쉬운 게

딥디크
오데썽오드뚜왈렛
30대
여
5
마치 울창한 숲 속에 있는 것처럼 시원하고 무거운데... 이상하게 상큼한 느낌이 남. 누군가는 이 향을 산사의 향 냄새 같다고 표현하던데... 비슷한 듯도 하지만 매캐하지 않은, 뭔가 신선한 느낌이 더 강함. 조금 시간이 지나면 오렌지 향이 은은히 퍼짐. 진하지 않은 과일 향이 꽤 오랜 시간 잔향으로 남음.
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '30대', '여', 5, '마치 울창한 숲 속에 있는 것처럼 시원하고 무거운데... 이상하게 상큼한 느낌이 남. 누군가는 이 향을 산사의 향 냄새 같다고 표현하던데... 비슷한 듯도 하지만 매캐하지 않은, 뭔가 신선한 느낌이 더 강함. 조금 시간이 지나면 오렌지 향이 은은히 퍼짐. 진하지 않은 과일 향이 꽤 오랜 시간 잔향으로 남음.')
딥디크
오데썽오드뚜왈렛


5
만족향이 너무너무너무 죠아여♥️
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족향이 너무너무너무 죠아여♥️')
딥디크
오데썽오드뚜왈렛
20대
남
5
첫향이 정말 오렌지 껍질 같은 느낌이 많이 들고요! 잔향도 다른 딥디크 향수랑 다르게 오래가고 은은하니 좋네요!
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '20대', '남', 5, '첫향이 정말 오렌지 껍질 같은 느낌이 많이 들고요! 잔향도 다른 딥디크 향수랑 다르게 오래가고 은은하니 좋네요!')
39
딥디크
오데썽오드뚜왈렛


5
만족너무좋으네요
**************************************************
.('딥디크', '오데썽오드뚜왈렛', '', '', 5, '만족너무좋으네요')
딥디크
오데썽오드뚜왈렛
20대
여
5
역시 딥디크는 짱!!
**********

In [11]:
from selenium import webdriver
from bs4 import BeautifulSoup as bsp
from selenium.webdriver.chrome.options import Options

CHROMEDRIVER_PATH = 'C:/asi/project/chromedriver_win32 (3)/chromedriver' # Windows는 chromedriver_78.exe
WINDOW_SIZE = "1920,1080"
chrome_options = Options()
chrome_options.add_argument(f"--window-size={ WINDOW_SIZE }")
chrome_options.add_argument('Content-Type=application/json; charset=utf-8')
driver = webdriver.Chrome( executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options )
url = ('https://perfumegraphy.com')
driver.get(url)
#driver = webdriver.Chrome('C:/asi/project/chromedriver_win32 (3)/chromedriver')
#url = ('https://perfumegraphy.com/product/list.html?cate_no=201')
#driver.get(url)
driver.implicitly_wait(3)
driver.find_element_by_xpath('//*[@id="slideCateList"]/div/div').click()
driver.implicitly_wait(3)
time.sleep(3)
html = driver.page_source
src = bsp(html,'html.parser')
brands=src.find_all(attrs={'class':'-brand'})
for brand in brands:
    brand_url="https://perfumegraphy.com"+brand.find('a')['href']
    

<ipython-input-11-34f02ac0208d>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome( executable_path=CHROMEDRIVER_PATH, chrome_options=chrome_options )


https://perfumegraphy.com/product/list.html?cate_no=312
https://perfumegraphy.com/product/list.html?cate_no=166
https://perfumegraphy.com/product/list.html?cate_no=201
https://perfumegraphy.com/product/list.html?cate_no=303
https://perfumegraphy.com/product/list.html?cate_no=252
https://perfumegraphy.com/product/list.html?cate_no=246
https://perfumegraphy.com/product/list.html?cate_no=164
https://perfumegraphy.com/product/list.html?cate_no=167
https://perfumegraphy.com/product/list.html?cate_no=224
https://perfumegraphy.com/product/list.html?cate_no=261
https://perfumegraphy.com/product/list.html?cate_no=245
https://perfumegraphy.com/product/list.html?cate_no=168
https://perfumegraphy.com/product/list.html?cate_no=169
https://perfumegraphy.com/product/list.html?cate_no=157
https://perfumegraphy.com/product/list.html?cate_no=170
https://perfumegraphy.com/product/list.html?cate_no=211
https://perfumegraphy.com/product/list.html?cate_no=315
https://perfumegraphy.com/product/list.html?cate